In [34]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2024-01-01'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' ",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data

In [35]:
hard_womens=hist_data_dog('HArd','Womens',1,2,datefilter)
hard_mens=hist_data_dog('HArd','Mens',1,2,datefilter)
clay_womens=hist_data_dog('Clay','Womens',1,2,datefilter)
clay_mens=hist_data_dog('Clay','Mens',1,2,datefilter)
temp=pd.concat([hard_womens,hard_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Loss'] = (temp['Fav_Win'] == 0).astype(int)
temp['Fav_Loss_Count'] = temp.groupby('Fav')['Fav_Loss'].cumsum()
temp['Dog_Win'] = (temp['Fav_Win'] == 0).astype(int)
temp['Dog_Win_Count'] = temp.groupby('Dog')['Dog_Win'].cumsum()


In [36]:
temp
date_filter=temp[temp['Date']>'2025-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.7].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>2]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>2]
oddsfilter=oddsfilter[oddsfilter['Elo_Fav']==oddsfilter['Dog']]
#oddsfilter=oddsfilter[oddsfilter['Dog_Top100']>.5]
#oddsfilter=oddsfilter[oddsfilter['Fav_Top100']<.5]
#oddsfilter=oddsfilter[oddsfilter['dog_rank']<200]
#oddsfilter=oddsfilter[oddsfilter['fav_rank']<300]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Womens']
len(oddsfilter[oddsfilter['Fav_Loss']==1])/len(oddsfilter),len(oddsfilter)
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter

Total Profit: 1961.0 43 0.7209302325581395


,Date,Winner_Odds,Winner,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Games,Dog,Dog_Games,...,Dog_Top100,fav_rank_high,dog_rank_high,Fav_Win,DogWin,Fav_Loss,Fav_Loss_Count,Dog_Win,Dog_Win_Count,Profit
6348,2025-01-02,1.92,Alycia Parks,1.87,1.92,Greet Minnen,Alycia Parks,28,Alycia Parks,19,...,0.2,59.0,40.0,False,True,1,9,1,6,92.0
6398,2025-01-04,2.01,Alycia Parks,1.78,2.01,Katie Volynets,Alycia Parks,22,Alycia Parks,20,...,0.2,56.0,40.0,False,True,1,6,1,7,101.0
6401,2025-01-04,1.95,Polina Kudermetova,1.84,1.95,Anhelina Kalinina,Polina Kudermetova,12,Polina Kudermetova,10,...,0.6,25.0,106.0,False,True,1,7,1,7,95.0
6432,2025-01-05,1.98,Katerina Siniakova,1.77,1.98,Ashlyn Krueger,Katerina Siniakova,40,Katerina Siniakova,35,...,0.5,51.0,27.0,False,True,1,7,1,6,98.0
6455,2025-01-05,1.9,Nuria Parrizas Diaz,1.85,1.90,Arantxa Rus,Nuria Parrizas Diaz,18,Nuria Parrizas Diaz,13,...,0.8,41.0,45.0,False,True,1,6,1,7,90.0
6459,2025-01-05,1.84,Ann Li,1.84,1.91,Ann Li,Suzan Lamens,12,Suzan Lamens,18,...,0.5,44.0,86.0,True,False,0,3,0,7,-100.0
6623,2025-01-07,1.82,Renata Zarazua,1.82,1.96,Renata Zarazua,Lucia Bronzetti,11,Lucia Bronzetti,33,...,0.3,51.0,46.0,True,False,0,4,0,7,-100.0
6716,2025-01-08,1.94,Varvara Lepchenko,1.83,1.94,Heather Watson,Varvara Lepchenko,13,Varvara Lepchenko,13,...,0.3,38.0,19.0,False,True,1,6,1,6,94.0
6717,2025-01-08,2.08,Leolia Jeanjean,1.71,2.08,Ella Seidel,Leolia Jeanjean,10,Leolia Jeanjean,11,...,0.5,117.0,102.0,False,True,1,4,1,4,108.0
6729,2025-01-08,1.82,Veronika Kudermetova,1.82,1.97,Veronika Kudermetova,Camila Osorio,30,Camila Osorio,26,...,0.4,9.0,33.0,True,False,0,8,0,3,-100.0


In [37]:
date_filter=temp[temp['Date']>'2023-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']<1.7].copy()
#oddsfilter=oddsfilter[oddsfilter['Fav_Odds']>1.4]
#oddsfilter=oddsfilter[oddsfilter['Fav_Odds']<1.8].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>5]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>5]
oddsfilter=oddsfilter[oddsfilter['Elo_Fav']==oddsfilter['Dog']]
#oddsfilter=oddsfilter[oddsfilter['Dog_Top100']>.5]
#oddsfilter=oddsfilter[oddsfilter['Fav_Top100']<.5]
oddsfilter=oddsfilter[oddsfilter['dog_rank']>200]
oddsfilter=oddsfilter[oddsfilter['fav_rank']>100]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
len(oddsfilter[oddsfilter['Fav_Loss']==1])/len(oddsfilter),len(oddsfilter)
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter

Total Profit: 2320.0 36 0.6666666666666666


,Date,Winner_Odds,Winner,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Games,Dog,Dog_Games,...,Dog_Top100,fav_rank_high,dog_rank_high,Fav_Win,DogWin,Fav_Loss,Fav_Loss_Count,Dog_Win,Dog_Win_Count,Profit
2756,2024-07-10,2.88,Philip Sekulic,1.38,2.88,Mark Lajal,Philip Sekulic,24,Philip Sekulic,11,...,0.2,191.0,238.0,False,True,1,6,1,8,188.0
3140,2024-08-08,2.53,Tristan Schoolkate,1.48,2.53,Harold Mayot,Tristan Schoolkate,10,Tristan Schoolkate,13,...,0.0,108.0,176.0,False,True,1,6,1,7,153.0
3281,2024-08-13,2.64,Brandon Holt,1.46,2.64,Zachary Svajda,Brandon Holt,12,Brandon Holt,14,...,0.4,109.0,180.0,False,True,1,11,1,8,164.0
3602,2024-08-22,2.41,Antoine Escoffier,1.54,2.41,Benjamin Bonzi,Antoine Escoffier,13,Antoine Escoffier,16,...,0.4,42.0,135.0,False,True,1,10,1,7,141.0
3667,2024-08-23,2.46,Rio Noguchi,1.51,2.46,Yuta Shimizu,Rio Noguchi,14,Rio Noguchi,15,...,0.0,203.0,201.0,False,True,1,7,1,9,146.0
4353,2024-09-26,1.49,Harold Mayot,1.49,2.48,Harold Mayot,Antoine Escoffier,12,Antoine Escoffier,16,...,0.4,108.0,135.0,True,False,0,8,0,7,-100.0
4842,2024-10-10,1.53,Benjamin Bonzi,1.53,2.38,Benjamin Bonzi,Clement Chidekh,18,Clement Chidekh,10,...,0.0,42.0,217.0,True,False,0,13,0,6,-100.0
4888,2024-10-12,2.54,Bernard Tomic,1.48,2.54,Alexis Galarneau,Bernard Tomic,22,Bernard Tomic,16,...,0.0,153.0,17.0,False,True,1,7,1,7,154.0
4924,2024-10-13,2.4,Bernard Tomic,1.53,2.40,Tristan Schoolkate,Bernard Tomic,24,Bernard Tomic,12,...,0.0,167.0,17.0,False,True,1,7,1,8,140.0
5602,2024-10-30,1.3,Alexis Galarneau,1.30,3.39,Alexis Galarneau,Naoki Nakagawa,11,Naoki Nakagawa,11,...,0.0,153.0,334.0,True,False,0,8,0,6,-100.0


In [38]:
# Load the dataset
df = temp[(temp['Sex']!='Wobmens')&(temp['Fav_Odds']<2)&(temp['Fav_Odds']>1.4)]


# Feature selection and preprocessing
features = ['fav_percent', 'dog_percent', 'Elo_Fav_Elo', 'Elo_Dog_Elo', 
            'fav_rank', 'dog_rank', 'dog_rank_high', 'fav_rank_high','Fav_Top100','Dog_Top100','Dog_Win_Count','Fav_Loss_Count']

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Drop the 'Winner_Odds' column
df = df.drop(columns=['Winner_Odds'])


target = 'Fav_Win'

X = df[features]  # Features for training
y = df[target]  # Target variable

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42,shuffle=False)

# Model training
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Prediction and evaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Add predictions and actuals to the test set DataFrame
test_with_predictions = X_test.copy()
test_with_predictions['Actual'] = y_test
test_with_predictions['Predicted'] = y_pred

# Add Fav_Odds and Dog_Odds back to the test set for profit calculation
test_with_predictions = test_with_predictions.merge(
    df[['Date','Fav','Dog','Fav_Odds', 'Dog_Odds']], left_index=True, right_index=True
)

# Define the stake amount
stake = 100  # Example stake amount

# Profit calculation
def calculate_profit(row):
    if row['Predicted'] == 1:  # If the model predicts the favorite wins
        if row['Actual'] == 1:  # If the prediction is correct
            return stake * (row['Fav_Odds'] - 1)  # Profit based on favorite odds
        else:
            return -stake  # Loss of the stake
    else:  # If the model predicts the underdog wins
        if row['Actual'] == 0:  # If the prediction is correct
            return stake * (row['Dog_Odds'] - 1)  # Profit based on underdog odds
        else:
            return -stake  # Loss of the stake

test_with_predictions['Profit'] = test_with_predictions.apply(calculate_profit, axis=1)

# Save the output DataFrame to clipboard
fav_predictions=test_with_predictions[test_with_predictions['Predicted']==1].copy()
# Calculate overall profit
fav_profit = fav_predictions['Profit'].sum()
print(f"Fav Profit: ${fav_profit:.2f}",len(fav_predictions))

dog_predictions=test_with_predictions[test_with_predictions['Predicted']==0].copy()
# Calculate overall profit
dog_profit = dog_predictions['Profit'].sum()
print(f"Dog Profit: ${dog_profit:.2f}", len(dog_predictions))
dog_predictions.to_clipboard()
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Train a random forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Get feature importances
feature_importances = rf_model.feature_importances_

# Sort and display
sorted_indices = np.argsort(feature_importances)[::-1]
for i in sorted_indices:
    print(f"{features[i]}: {feature_importances[i]:.4f}")


Accuracy: 0.5597147950089126
              precision    recall  f1-score   support

       False       0.49      0.68      0.57       242
        True       0.66      0.47      0.55       319

    accuracy                           0.56       561
   macro avg       0.58      0.57      0.56       561
weighted avg       0.59      0.56      0.56       561

Fav Profit: $1342.00 228
Dog Profit: $3705.00 333
Dog_Win_Count: 0.1510
Fav_Loss_Count: 0.1397
Elo_Fav_Elo: 0.0948
Elo_Dog_Elo: 0.0941
fav_rank: 0.0818
dog_rank: 0.0811
dog_rank_high: 0.0780
fav_rank_high: 0.0761
fav_percent: 0.0727
dog_percent: 0.0700
Fav_Top100: 0.0308
Dog_Top100: 0.0299


In [40]:
import numpy as np

# Define the odds bands (10c increments from 1.00 to 2.00)
bins = np.arange(1.0, 2.1, 0.1)  # Create bins: [1.0, 1.1, ..., 2.0]
labels = [f"{bins[i]:.1f} to {bins[i+1]:.1f}" for i in range(len(bins) - 1)]

# Create a new column for odds bands
dog_predictions['Fav_Odds_Band'] = pd.cut(dog_predictions['Fav_Odds'], bins=bins, labels=labels, right=False)

# Add a column to indicate if the prediction was correct
dog_predictions['Correct_Prediction'] = dog_predictions['Actual'] == dog_predictions['Predicted']

# Group by Fav_Odds_Band and calculate Profit, count of records, and winning percentage
grouped_dog = dog_predictions.groupby('Fav_Odds_Band').agg(
    Total_Profit=('Profit', 'sum'),
    Record_Count=('Profit', 'count'),
    Winning_Percentage=('Correct_Prediction', lambda x: (x.sum() / len(x) * 100) if len(x) > 0 else 0)
).reset_index()

# Print the results
print(grouped_dog)



import numpy as np

# Define the odds bands (10c increments from 1.00 to 2.00)
bins = np.arange(1.0, 2.1, 0.1)  # Create bins: [1.0, 1.1, ..., 2.0]
labels = [f"{bins[i]:.1f} to {bins[i+1]:.1f}" for i in range(len(bins) - 1)]

# Create a new column for odds bands
fav_predictions['Fav_Odds_Band'] = pd.cut(fav_predictions['Fav_Odds'], bins=bins, labels=labels, right=False)

# Add a column to indicate if the prediction was correct
fav_predictions['Correct_Prediction'] = fav_predictions['Actual'] == fav_predictions['Predicted']

# Group by Fav_Odds_Band and calculate Profit, count of records, and winning percentage
grouped_fav = fav_predictions.groupby('Fav_Odds_Band').agg(
    Total_Profit=('Profit', 'sum'),
    Record_Count=('Profit', 'count'),
    Winning_Percentage=('Correct_Prediction', lambda x: (x.sum() / len(x) * 100) if len(x) > 0 else 0)
).reset_index()

# Print the results
print(grouped_fav)

  Fav_Odds_Band  Total_Profit  Record_Count  Winning_Percentage
0    1.0 to 1.1           0.0             0            0.000000
1    1.1 to 1.2           0.0             0            0.000000
2    1.2 to 1.3           0.0             0            0.000000
3    1.3 to 1.4           0.0             0            0.000000
4    1.4 to 1.5        1081.0            75           42.666667
5    1.5 to 1.6         792.0            87           45.977011
6    1.6 to 1.7         477.0            69           49.275362
7    1.7 to 1.8        1672.0            64           62.500000
8    1.8 to 1.9        -317.0            38           47.368421
9    1.9 to 2.0           0.0             0            0.000000
  Fav_Odds_Band  Total_Profit  Record_Count  Winning_Percentage
0    1.0 to 1.1           0.0             0            0.000000
1    1.1 to 1.2           0.0             0            0.000000
2    1.2 to 1.3           0.0             0            0.000000
3    1.3 to 1.4           0.0           

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9548\2049959661.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_dog = dog_predictions.groupby('Fav_Odds_Band').agg(
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9548\2049959661.py:38: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_fav = fav_predictions.groupby('Fav_Odds_Band').agg(


In [41]:
# Current date and time
current_datetime = datetime.datetime.now()

# Convert to string
date_string = current_datetime.strftime("%Y-%m-%d")

today=pd.read_pickle('./Hard_Today')
today=today[today['Fav_Odds']>1.4]
today_features=today[features]
today['Predicted']=model.predict(today_features)
probabilities = model.predict_proba(today_features)

today['Predicted_Prob'] = probabilities[:, 0]  # Column 1 corresponds to the positive class
today['Date']=date_string
dog=today[today['Predicted']==False][['Date','Time','Dog','Predicted','Predicted_Prob','Dog_Odds','Fav_Odds','Fav']]
dog = dog[dog['Dog_Odds'] >1]
#dog = dog[dog['Predicted_Prob']>0.6]
#dog = dog[dog['Predicted_Prob']<0.7]
dog

,Date,Time,Dog,Predicted,Predicted_Prob,Dog_Odds,Fav_Odds,Fav
0,2025-03-19,01:00,Viktoriya Tomova,False,0.55,2.25,1.64,Caty Mcnally
4,2025-03-19,01:05,Alexander Blockx,False,0.50,1.89,1.86,Terence Atmane
5,2025-03-19,01:05,Varvara Gracheva,False,0.56,1.98,1.82,Alycia Parks
6,2025-03-19,01:10,Thiago Agustin Tirante,False,0.68,2.13,1.68,Alexander Shevchenko
7,2025-03-19,02:40,Rei Sakamoto,False,0.51,2.32,1.58,James Duckworth
8,2025-03-19,02:40,Camilo Ugo Carabelli,False,0.53,2.39,1.55,Ethan Quinn
9,2025-03-19,02:45,Darwin Blanch,False,0.65,2.54,1.49,Chun Hsin Tseng
11,2025-03-19,03:20,Jessica Bouzas Maneiro,False,0.59,2.27,1.63,Lucia Bronzetti
15,2025-03-19,04:30,Tristan Schoolkate,False,0.57,2.80,1.42,Adam Walton
17,2025-03-19,05:05,Shintaro Mochizuki,False,0.61,2.28,1.59,Francisco Comesana


In [42]:
fav=today[today['Predicted']==True][['Date','Time','Fav','Predicted','Predicted_Prob','Dog_Odds','Fav_Odds','Dog']]
fav = fav[fav['Dog_Odds'] >1]
#fav = fav[fav['Predicted_Prob']>0.1]
fav

,Date,Time,Fav,Predicted,Predicted_Prob,Dog_Odds,Fav_Odds,Dog
1,2025-03-19,01:05,Anastasia Pavlyuchenkova,True,0.46,2.33,1.60,Magda Linette
2,2025-03-19,01:05,Mackenzie Mcdonald,True,0.41,2.60,1.47,Pavel Kotov
18,2025-03-19,05:05,Gabriel Diallo,True,0.45,2.53,1.50,Brandon Holt
20,2025-03-19,05:45,Gabriela Ruse Elena,True,0.49,1.92,1.87,Polina Kudermetova
23,2025-03-19,08:10,Olga Danilovic,True,0.49,1.97,1.83,Hailey Baptiste
25,2025-03-19,08:45,Taylor Townsend,True,0.46,1.95,1.83,Ann Li


In [43]:
fav.to_sql('DailyPredictions_Fav',con=devengine,index=False,if_exists='append')
dog.to_sql('DailyPredictions_Dog',con=devengine,index=False,if_exists='append')
predictions=pd.read_sql_table('DailyPredictions_Dog',con=devengine)
results=temp
merged_df = pd.merge(
    predictions, 
    results[['Date', 'Fav', 'Dog', 'Winner']],  # Select only the Winner column from results
    on=['Date', 'Fav', 'Dog'], 
    how='left'  # Use left join to retain all rows from predictions
)
merged_df_filtered=merged_df[merged_df['Winner'].notna()].copy()
merged_df_filtered=merged_df[merged_df['Dog_Odds']>2.2]
#merged_df_filtered=merged_df_filtered[merged_df_filtered['Predicted_Prob']>0.55]
merged_df_filtered=merged_df_filtered[merged_df_filtered['Predicted_Prob']<.55]
merged_df_filtered['Profit'] = merged_df_filtered.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = merged_df_filtered['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(merged_df_filtered),len(merged_df_filtered[merged_df_filtered['Dog']==merged_df_filtered['Winner']])/len(merged_df_filtered))


Total Profit: -440.0 58 0.3793103448275862


In [44]:
predictions=pd.read_sql_table('DailyPredictions_Fav',con=devengine)
results=temp
merged_df = pd.merge(
    predictions, 
    results[['Date', 'Fav', 'Dog', 'Winner']],  # Select only the Winner column from results
    on=['Date', 'Fav', 'Dog'], 
    how='left'  # Use left join to retain all rows from predictions
)
merged_df_filtered=merged_df[merged_df['Winner'].notna()].copy()
merged_df_filtered=merged_df_filtered[merged_df_filtered['Predicted_Prob']<0.1]
#merged_df_filtered=merged_df_filtered[merged_df_filtered['Date']=='2025-02-20']
merged_df_filtered['Profit'] = merged_df_filtered.apply(
    lambda row: (row['Dog_Odds'] * stake)-stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = merged_df_filtered['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(merged_df_filtered),len(merged_df_filtered[merged_df_filtered['Dog']==merged_df_filtered['Winner']])/len(merged_df_filtered))

predictions=pd.read_sql_table('DailyPredictions_Fav',con=devengine)
results=temp
merged_df = pd.merge(
    predictions, 
    results[['Date', 'Fav', 'Dog', 'Winner']],  # Select only the Winner column from results
    on=['Date', 'Fav', 'Dog'], 
    how='left'  # Use left join to retain all rows from predictions
)
merged_df_filtered=merged_df[merged_df['Winner'].notna()].copy()
merged_df_filtered=merged_df_filtered[merged_df_filtered['Predicted_Prob']>0.1]
merged_df_filtered['Profit'] = merged_df_filtered.apply(
    lambda row: (row['Fav_Odds'] * stake)-stake if row['Winner'] == row['Fav'] else -stake, axis=1
)

# Calculate total profit
total_profit = merged_df_filtered['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(merged_df_filtered),len(merged_df_filtered[merged_df_filtered['Fav']==merged_df_filtered['Winner']])/len(merged_df_filtered))
#merged_df[merged_df['Date']=='2025-02-27']

Total Profit: 1021.0 51 0.5294117647058824
Total Profit: -687.0 118 0.5847457627118644


In [45]:
today=pd.read_pickle('.\Hard_Today')
today=today[today['Fav_Odds']>1.7]
today[today['Sex']=='Womens']

<>:1: SyntaxWarning: invalid escape sequence '\H'
<>:1: SyntaxWarning: invalid escape sequence '\H'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9548\2751913006.py:1: SyntaxWarning: invalid escape sequence '\H'
  today=pd.read_pickle('.\Hard_Today')


,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,...,Fav_Top100,Dog_Top100,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count
5,01:05,1.82,1.98,Alycia Parks,Alycia Parks,38%,29,Varvara Gracheva,28%,29,...,0.4,0.2,0.73,0.24,0.54,0.30,0.6,0.4,8.0,9.0
19,05:20,1.71,2.13,Claire Liu,Claire Liu,100%,2,Linda Fruhvirtova,67%,3,...,0.5,0.3,0.63,0.37,0.66,0.35,0.8,0.4,8.0,7.0
20,05:45,1.87,1.92,Gabriela Ruse Elena,Gabriela Ruse Elena,33%,9,Polina Kudermetova,50%,16,...,0.3,0.4,NaN,NaN,0.00,0.00,0.6,0.4,5.0,11.0
23,08:10,1.83,1.97,Olga Danilovic,Hailey Baptiste,56%,9,Hailey Baptiste,42%,26,...,0.5,0.4,0.71,0.31,0.64,0.29,0.4,0.6,6.0,12.0
25,08:45,1.83,1.95,Taylor Townsend,Taylor Townsend,58%,26,Ann Li,50%,18,...,0.5,0.4,0.64,0.29,0.64,0.31,0.6,0.6,6.0,1.0


In [46]:
today=pd.read_pickle('.\Hard_Today')
today=today[today['Fav_Odds']<1.7]
today=today[today['Fav_Loss_Count']>5]
today=today[today['Dog_Win_Count']>5]
today=today[today['Elo_Fav']==today['Dog']]
today=today[today['fav_rank']>100]
today=today[today['dog_rank']>200]
today[today['Sex']!='Womens']

<>:1: SyntaxWarning: invalid escape sequence '\H'
<>:1: SyntaxWarning: invalid escape sequence '\H'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9548\4104527065.py:1: SyntaxWarning: invalid escape sequence '\H'
  today=pd.read_pickle('.\Hard_Today')


,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,...,Fav_Top100,Dog_Top100,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count
